<a href="https://colab.research.google.com/github/mesh98a/DeepLearning/blob/main/labN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuer Abschnitt